In [1]:
# === Librairies pour la manipulation et l'analyse de données ===
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

# === Librairies pour la visualisation ===
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# === Librairies pour l'affichage interactif (Jupyter/IPython) ===
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option("display.width", None)


In [2]:
user = os.getlogin().lower()
print(f"Detected user: {user}")

# Use the default path only if the username is 'rabah'
if user == "rabah":
    file_path = r"A:\OneDrive - Sopex London Ltd\Shared\19. Shipments - Sopex\New_Freight_file_V1.xlsx"
    
else:
    # Fallback logic for other users
    file_path = r"C:\Users\djenn\OneDrive - Sopex London Ltd\Shared\19. Shipments - Sopex\New_Freight_file_V1.xlsx"

Detected user: djenn


In [3]:
df= pd.read_excel(file_path, sheet_name="Freight")

display(df.head())
print(df.columns.tolist())

,Port POL,Country POD,Port POD,Freight_All_In_Karen,Freight,Freight_Currency,Surcharge,Surcharge_Currency,Extra,Unit,Extra_Currency,Terminal POL,Shipping Line,Shipping Line2,Free Time POL,Free Time POD,Validity,Standardized,Region,FREIGHT_USE_USD,Freight_Surcharge,EXTRA_USD,EXTRA_USD_TONNES,FREIGHT_ALL_USD,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,EUR/USD,1.19
0,Santos,Tanzania,Dar Es Salaam,2800.0,2800.0,USD,0.0,USD,NaN,NaN,NaN,NaN,Windlog,NaN,14,14,January 2025,2025-01-31,S & E Africa,2800.0,0.0,0.0,0.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Santos,Benin,Cotonou,1665.0,1665.0,USD,0.0,USD,NaN,NaN,NaN,NaN,Windlog,NaN,14,14,January 2025,2025-01-31,Africa West,1665.0,0.0,0.0,0.0,1665.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Santos,UAE,Jebel Ali,1775.0,1775.0,USD,0.0,USD,NaN,NaN,NaN,NaN,Windlog,NaN,14,14,January 2025,2025-01-31,Middle East,1775.0,0.0,0.0,0.0,1775.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Santos,Turkey,Mersin,1350.0,1350.0,USD,0.0,USD,NaN,NaN,NaN,NaN,Windlog,NaN,14,14,January 2025,2025-01-31,Mediterranean,1350.0,0.0,0.0,0.0,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Santos,Ghana,Tema,1700.0,1700.0,USD,0.0,USD,NaN,NaN,NaN,NaN,Windlog,NaN,14,14,January 2025,2025-01-31,Africa West,1700.0,0.0,0.0,0.0,1700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['Port POL', 'Country POD', 'Port POD', 'Freight_All_In_Karen', 'Freight', 'Freight_Currency', 'Surcharge', 'Surcharge_Currency', 'Extra', 'Unit', 'Extra_Currency', 'Terminal POL', 'Shipping Line', 'Shipping Line2', 'Free Time POL', 'Free Time POD', 'Validity', 'Standardized', 'Region', 'FREIGHT_USE_USD', 'Freight_Surcharge', 'EXTRA_USD', 'EXTRA_USD_TONNES', 'FREIGHT_ALL_USD', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'EUR/USD', 1.19]


In [4]:
freight= df[['Port POL', 'Country POD', 'Port POD', 
             'Freight_All_In_Karen', 
             'Shipping Line',
             'Standardized', 'Region']].copy()

display(freight.tail())
print(freight.info())

,Port POL,Country POD,Port POD,Freight_All_In_Karen,Shipping Line,Standardized,Region
1179,Jabel Ali,Singapore,Singapore,282.03,Hapag Lloyd Web,2026-03-31,Far East / South East Asia
1180,Dunkirk,Turkey,Mersin,1150.73,Manuport,2026-03-31,Mediterranean
1181,Antwerp,Turkey,Mersin,329.63,Manuport,2026-03-31,Mediterranean
1182,Santos,Costa Rica,Moin,1015.00,Maersk SPOT,2026-03-31,Americas
1183,Santos,Costa Rica,Moin,995.00,Maersk SPOT,2026-03-31,Americas


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184 entries, 0 to 1183
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Port POL              1184 non-null   object        
 1   Country POD           1184 non-null   object        
 2   Port POD              1184 non-null   object        
 3   Freight_All_In_Karen  1184 non-null   float64       
 4   Shipping Line         1176 non-null   object        
 5   Standardized          980 non-null    datetime64[ns]
 6   Region                1177 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 64.9+ KB
None


### Working on valid dates

In [5]:
df_clean = freight.dropna(subset=['Standardized'])

latest = df_clean.loc[df_clean.groupby(['Port POL', 'Port POD'])['Standardized'].idxmax()]


today = datetime.today()

# Make a real copy to avoid SettingWithCopyWarning
df_future = df_clean[df_clean['Standardized'] > today].copy()

display(df_future.head())
print(df_future.info())


,Port POL,Country POD,Port POD,Freight_All_In_Karen,Shipping Line,Standardized,Region
981,Santos,Israel,Ashdod,1647.0,Monero,2026-02-28,Mediterranean
982,Santos,Lebanon,Beirut,1927.0,Monero,2026-02-28,Mediterranean
983,Santos,Albania,Durres,1647.0,Monero,2026-02-28,Europe
984,Santos,Turkey,Mersin,1747.0,Monero,2026-02-28,Mediterranean
985,Santos,Senegal,Dakar,1627.0,Monero,2026-02-28,Africa West


<class 'pandas.core.frame.DataFrame'>
Index: 196 entries, 981 to 1183
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Port POL              196 non-null    object        
 1   Country POD           196 non-null    object        
 2   Port POD              196 non-null    object        
 3   Freight_All_In_Karen  196 non-null    float64       
 4   Shipping Line         195 non-null    object        
 5   Standardized          196 non-null    datetime64[ns]
 6   Region                196 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 12.2+ KB
None


In [7]:
import pandas as pd
import numpy as np

# --- 1) Clean copy & strip string columns ---
df_future = df_future.copy()
for col in ['Port POL', 'Country POD', 'Port POD', 'Region']:
    df_future[col] = df_future[col].astype(str).str.strip()

# --- 2) Pivot: min Freight per POL for each (Region, Country POD, Port POD) ---
pivot = df_future.pivot_table(
    index=['Region', 'Country POD', 'Port POD'],
    columns='Port POL',
    values='Freight_All_In_Karen',
    aggfunc='min'
).round(0)

# Sort rows and columns for readability
pivot = pivot.sort_index().sort_index(axis=1)

# --- Remove index name so it doesn't appear as a column ---
pivot.index.name = None

# Remove column axis name
pivot.columns.name = None

# Flatten the index into columns
pivot_flat = pivot.reset_index()

display(pivot_flat.head())

print(pivot_flat.columns.tolist())

,Region,Country POD,Port POD,Antwerp,Buenaventura,Dunkirk,Jabel Ali,Laem Chabang,Le Havre,Mundra,Santos
0,Africa West,Angola,Cabinda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2851.0
1,Africa West,Angola,Lobito,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2351.0
2,Africa West,Angola,Luanda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1251.0
3,Africa West,Benin,Cotonou,1731.0,NaN,NaN,2956.0,1531.0,NaN,2317.0,1527.0
4,Africa West,Cameroon,Douala,1952.0,NaN,NaN,3998.0,2031.0,NaN,3865.0,1651.0


['Region', 'Country POD', 'Port POD', 'Antwerp', 'Buenaventura', 'Dunkirk', 'Jabel Ali', 'Laem Chabang', 'Le Havre', 'Mundra', 'Santos']


In [8]:
pivot_flat = pivot_flat[['Region', 'Country POD','Port POD', 'Santos', 'Mundra','Antwerp', 'Jabel Ali', 'Laem Chabang']].copy()

display(pivot_flat.head())
print(pivot_flat.info())

,Region,Country POD,Port POD,Santos,Mundra,Antwerp,Jabel Ali,Laem Chabang
0,Africa West,Angola,Cabinda,2851.0,NaN,NaN,NaN,NaN
1,Africa West,Angola,Lobito,2351.0,NaN,NaN,NaN,NaN
2,Africa West,Angola,Luanda,1251.0,NaN,NaN,NaN,NaN
3,Africa West,Benin,Cotonou,1527.0,2317.0,1731.0,2956.0,1531.0
4,Africa West,Cameroon,Douala,1651.0,3865.0,1952.0,3998.0,2031.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Region        48 non-null     object 
 1   Country POD   48 non-null     object 
 2   Port POD      48 non-null     object 
 3   Santos        43 non-null     float64
 4   Mundra        19 non-null     float64
 5   Antwerp       24 non-null     float64
 6   Jabel Ali     18 non-null     float64
 7   Laem Chabang  22 non-null     float64
dtypes: float64(5), object(3)
memory usage: 3.1+ KB
None


In [9]:
ports_pod_select = [
    "Ashdod",
    "Beirut",
    "Durres",
    "Mersin",
    "Conakry",
    "Dakar",
    "Douala",
    "Cotonou",
    "Lome",
    "Abidjan",
    "Tema",
    "Djibouti",
    "Mombasa",
    "Dar Es Salaam",
    "Tamatave",
    "Jebel Ali",
    "Aden",
    "Aqaba",
    "Colombo",
    "Singapore"
]


In [10]:
pivot_ports = pivot_flat[pivot_flat["Port POD"].isin(ports_pod_select)].copy()

pivot_ports = pivot_ports.sort_values(["Region", "Country POD", "Port POD"])

pivot_ports.loc[:, pivot_ports.select_dtypes("number").columns] = (pivot_ports.select_dtypes("number").replace(0, np.nan))

display(pivot_ports)

,Region,Country POD,Port POD,Santos,Mundra,Antwerp,Jabel Ali,Laem Chabang
3,Africa West,Benin,Cotonou,1527.0,2317.0,1731.0,2956.0,1531.0
4,Africa West,Cameroon,Douala,1651.0,3865.0,1952.0,3998.0,2031.0
7,Africa West,Ghana,Tema,1451.0,2725.0,1214.0,3245.0,1599.0
8,Africa West,Guinea,Conakry,3851.0,7204.0,4869.0,6088.0,4982.0
9,Africa West,Ivory Coast,Abidjan,1627.0,2404.0,1080.0,3012.0,1531.0
12,Africa West,Senegal,Dakar,1627.0,3599.0,1876.0,3018.0,2781.0
15,Africa West,Togo,Lome,1490.0,2370.0,1589.0,2891.0,1531.0
19,Europe,Albania,Durres,1647.0,2557.0,1292.0,1909.0,5225.0
24,Far East / South East Asia,Singapore,Singapore,941.0,624.0,18.0,282.0,350.0
25,Far East / South East Asia,Sri Lanka,Colombo,750.0,1869.0,491.0,339.0,1817.0


In [11]:
freight_cols = pivot_ports.select_dtypes("number").columns

pivot_ports["Cheapest_POL"]   = pivot_ports[freight_cols].idxmin(axis=1)
#pivot_ports["Cheapest_Value"] = pivot_ports[freight_cols].min(axis=1)

display(pivot_ports)

,Region,Country POD,Port POD,Santos,Mundra,Antwerp,Jabel Ali,Laem Chabang,Cheapest_POL
3,Africa West,Benin,Cotonou,1527.0,2317.0,1731.0,2956.0,1531.0,Santos
4,Africa West,Cameroon,Douala,1651.0,3865.0,1952.0,3998.0,2031.0,Santos
7,Africa West,Ghana,Tema,1451.0,2725.0,1214.0,3245.0,1599.0,Antwerp
8,Africa West,Guinea,Conakry,3851.0,7204.0,4869.0,6088.0,4982.0,Santos
9,Africa West,Ivory Coast,Abidjan,1627.0,2404.0,1080.0,3012.0,1531.0,Antwerp
12,Africa West,Senegal,Dakar,1627.0,3599.0,1876.0,3018.0,2781.0,Santos
15,Africa West,Togo,Lome,1490.0,2370.0,1589.0,2891.0,1531.0,Santos
19,Europe,Albania,Durres,1647.0,2557.0,1292.0,1909.0,5225.0,Antwerp
24,Far East / South East Asia,Singapore,Singapore,941.0,624.0,18.0,282.0,350.0,Antwerp
25,Far East / South East Asia,Sri Lanka,Colombo,750.0,1869.0,491.0,339.0,1817.0,Jabel Ali


In [12]:
#Different ports: 
pivot_ports["LQW_Santos"]   = (pivot_ports["Santos"]   / 27).round(1)
pivot_ports["HQW_Santos"]   = (pivot_ports["Santos"]   / 26).round(1)
pivot_ports["HQW_India"]    = (pivot_ports["Mundra"]   / 27).round(1)
pivot_ports["LQW_India"]    = (pivot_ports["Mundra"]   / 27).round(1)
pivot_ports["HQW_Antwerp"]  = (pivot_ports["Antwerp"]  / 24).round(1)
pivot_ports["HQW_Jabel_Ali"]   = (pivot_ports["Jabel Ali"] / 26).round(1)
pivot_ports["HQW_Laem_Chabang"] = (pivot_ports["Laem Chabang"] / 26).round(1)


display(pivot_ports)



,Region,Country POD,Port POD,Santos,Mundra,Antwerp,Jabel Ali,Laem Chabang,Cheapest_POL,LQW_Santos,HQW_Santos,HQW_India,LQW_India,HQW_Antwerp,HQW_Jabel_Ali,HQW_Laem_Chabang
3,Africa West,Benin,Cotonou,1527.0,2317.0,1731.0,2956.0,1531.0,Santos,56.6,58.7,85.8,85.8,72.1,113.7,58.9
4,Africa West,Cameroon,Douala,1651.0,3865.0,1952.0,3998.0,2031.0,Santos,61.1,63.5,143.1,143.1,81.3,153.8,78.1
7,Africa West,Ghana,Tema,1451.0,2725.0,1214.0,3245.0,1599.0,Antwerp,53.7,55.8,100.9,100.9,50.6,124.8,61.5
8,Africa West,Guinea,Conakry,3851.0,7204.0,4869.0,6088.0,4982.0,Santos,142.6,148.1,266.8,266.8,202.9,234.2,191.6
9,Africa West,Ivory Coast,Abidjan,1627.0,2404.0,1080.0,3012.0,1531.0,Antwerp,60.3,62.6,89.0,89.0,45.0,115.8,58.9
12,Africa West,Senegal,Dakar,1627.0,3599.0,1876.0,3018.0,2781.0,Santos,60.3,62.6,133.3,133.3,78.2,116.1,107.0
15,Africa West,Togo,Lome,1490.0,2370.0,1589.0,2891.0,1531.0,Santos,55.2,57.3,87.8,87.8,66.2,111.2,58.9
19,Europe,Albania,Durres,1647.0,2557.0,1292.0,1909.0,5225.0,Antwerp,61.0,63.3,94.7,94.7,53.8,73.4,201.0
24,Far East / South East Asia,Singapore,Singapore,941.0,624.0,18.0,282.0,350.0,Antwerp,34.9,36.2,23.1,23.1,0.8,10.8,13.5
25,Far East / South East Asia,Sri Lanka,Colombo,750.0,1869.0,491.0,339.0,1817.0,Jabel Ali,27.8,28.8,69.2,69.2,20.5,13.0,69.9


In [13]:
pivot_ports_tons=pivot_ports.copy()
display(pivot_ports_tons)

,Region,Country POD,Port POD,Santos,Mundra,Antwerp,Jabel Ali,Laem Chabang,Cheapest_POL,LQW_Santos,HQW_Santos,HQW_India,LQW_India,HQW_Antwerp,HQW_Jabel_Ali,HQW_Laem_Chabang
3,Africa West,Benin,Cotonou,1527.0,2317.0,1731.0,2956.0,1531.0,Santos,56.6,58.7,85.8,85.8,72.1,113.7,58.9
4,Africa West,Cameroon,Douala,1651.0,3865.0,1952.0,3998.0,2031.0,Santos,61.1,63.5,143.1,143.1,81.3,153.8,78.1
7,Africa West,Ghana,Tema,1451.0,2725.0,1214.0,3245.0,1599.0,Antwerp,53.7,55.8,100.9,100.9,50.6,124.8,61.5
8,Africa West,Guinea,Conakry,3851.0,7204.0,4869.0,6088.0,4982.0,Santos,142.6,148.1,266.8,266.8,202.9,234.2,191.6
9,Africa West,Ivory Coast,Abidjan,1627.0,2404.0,1080.0,3012.0,1531.0,Antwerp,60.3,62.6,89.0,89.0,45.0,115.8,58.9
12,Africa West,Senegal,Dakar,1627.0,3599.0,1876.0,3018.0,2781.0,Santos,60.3,62.6,133.3,133.3,78.2,116.1,107.0
15,Africa West,Togo,Lome,1490.0,2370.0,1589.0,2891.0,1531.0,Santos,55.2,57.3,87.8,87.8,66.2,111.2,58.9
19,Europe,Albania,Durres,1647.0,2557.0,1292.0,1909.0,5225.0,Antwerp,61.0,63.3,94.7,94.7,53.8,73.4,201.0
24,Far East / South East Asia,Singapore,Singapore,941.0,624.0,18.0,282.0,350.0,Antwerp,34.9,36.2,23.1,23.1,0.8,10.8,13.5
25,Far East / South East Asia,Sri Lanka,Colombo,750.0,1869.0,491.0,339.0,1817.0,Jabel Ali,27.8,28.8,69.2,69.2,20.5,13.0,69.9


In [14]:
conv_cols = [
    "LQW_Santos",
    "HQW_Santos",
    "HQW_India",
    "LQW_India",
    "HQW_Antwerp",
    "HQW_Jabel_Ali",
    "HQW_Laem_Chabang"
]


In [15]:
pivot_ports_tons["Cheapest_Origin"] = (
    pivot_ports_tons[conv_cols]
    .idxmin(axis=1)
)

pivot_ports_tons["Cheapest_Origin_Value"] = (
    pivot_ports_tons[conv_cols]
    .min(axis=1)
)       


display(pivot_ports_tons)
print(pivot_ports_tons.columns.tolist())

,Region,Country POD,Port POD,Santos,Mundra,Antwerp,Jabel Ali,Laem Chabang,Cheapest_POL,LQW_Santos,HQW_Santos,HQW_India,LQW_India,HQW_Antwerp,HQW_Jabel_Ali,HQW_Laem_Chabang,Cheapest_Origin,Cheapest_Origin_Value
3,Africa West,Benin,Cotonou,1527.0,2317.0,1731.0,2956.0,1531.0,Santos,56.6,58.7,85.8,85.8,72.1,113.7,58.9,LQW_Santos,56.6
4,Africa West,Cameroon,Douala,1651.0,3865.0,1952.0,3998.0,2031.0,Santos,61.1,63.5,143.1,143.1,81.3,153.8,78.1,LQW_Santos,61.1
7,Africa West,Ghana,Tema,1451.0,2725.0,1214.0,3245.0,1599.0,Antwerp,53.7,55.8,100.9,100.9,50.6,124.8,61.5,HQW_Antwerp,50.6
8,Africa West,Guinea,Conakry,3851.0,7204.0,4869.0,6088.0,4982.0,Santos,142.6,148.1,266.8,266.8,202.9,234.2,191.6,LQW_Santos,142.6
9,Africa West,Ivory Coast,Abidjan,1627.0,2404.0,1080.0,3012.0,1531.0,Antwerp,60.3,62.6,89.0,89.0,45.0,115.8,58.9,HQW_Antwerp,45.0
12,Africa West,Senegal,Dakar,1627.0,3599.0,1876.0,3018.0,2781.0,Santos,60.3,62.6,133.3,133.3,78.2,116.1,107.0,LQW_Santos,60.3
15,Africa West,Togo,Lome,1490.0,2370.0,1589.0,2891.0,1531.0,Santos,55.2,57.3,87.8,87.8,66.2,111.2,58.9,LQW_Santos,55.2
19,Europe,Albania,Durres,1647.0,2557.0,1292.0,1909.0,5225.0,Antwerp,61.0,63.3,94.7,94.7,53.8,73.4,201.0,HQW_Antwerp,53.8
24,Far East / South East Asia,Singapore,Singapore,941.0,624.0,18.0,282.0,350.0,Antwerp,34.9,36.2,23.1,23.1,0.8,10.8,13.5,HQW_Antwerp,0.8
25,Far East / South East Asia,Sri Lanka,Colombo,750.0,1869.0,491.0,339.0,1817.0,Jabel Ali,27.8,28.8,69.2,69.2,20.5,13.0,69.9,HQW_Jabel_Ali,13.0


['Region', 'Country POD', 'Port POD', 'Santos', 'Mundra', 'Antwerp', 'Jabel Ali', 'Laem Chabang', 'Cheapest_POL', 'LQW_Santos', 'HQW_Santos', 'HQW_India', 'LQW_India', 'HQW_Antwerp', 'HQW_Jabel_Ali', 'HQW_Laem_Chabang', 'Cheapest_Origin', 'Cheapest_Origin_Value']


In [18]:
pivot_ports_tons_only =pivot_ports_tons[['Region', 'Country POD', 'Port POD',
                                    'LQW_Santos','LQW_India' ,'HQW_Santos', 'HQW_India','HQW_Antwerp', 'HQW_Jabel_Ali', 
                                    'HQW_Laem_Chabang', 'Cheapest_Origin', 'Cheapest_Origin_Value']].copy()

display(pivot_ports_tons_only)

,Region,Country POD,Port POD,LQW_Santos,LQW_India,HQW_Santos,HQW_India,HQW_Antwerp,HQW_Jabel_Ali,HQW_Laem_Chabang,Cheapest_Origin,Cheapest_Origin_Value
3,Africa West,Benin,Cotonou,56.6,85.8,58.7,85.8,72.1,113.7,58.9,LQW_Santos,56.6
4,Africa West,Cameroon,Douala,61.1,143.1,63.5,143.1,81.3,153.8,78.1,LQW_Santos,61.1
7,Africa West,Ghana,Tema,53.7,100.9,55.8,100.9,50.6,124.8,61.5,HQW_Antwerp,50.6
8,Africa West,Guinea,Conakry,142.6,266.8,148.1,266.8,202.9,234.2,191.6,LQW_Santos,142.6
9,Africa West,Ivory Coast,Abidjan,60.3,89.0,62.6,89.0,45.0,115.8,58.9,HQW_Antwerp,45.0
12,Africa West,Senegal,Dakar,60.3,133.3,62.6,133.3,78.2,116.1,107.0,LQW_Santos,60.3
15,Africa West,Togo,Lome,55.2,87.8,57.3,87.8,66.2,111.2,58.9,LQW_Santos,55.2
19,Europe,Albania,Durres,61.0,94.7,63.3,94.7,53.8,73.4,201.0,HQW_Antwerp,53.8
24,Far East / South East Asia,Singapore,Singapore,34.9,23.1,36.2,23.1,0.8,10.8,13.5,HQW_Antwerp,0.8
25,Far East / South East Asia,Sri Lanka,Colombo,27.8,69.2,28.8,69.2,20.5,13.0,69.9,HQW_Jabel_Ali,13.0


In [ ]:
print(pivot_ports_tons.columns.tolist())

# Example:
NY_Price  = 13.44 * 22.0462
LDN_Price = 396

# --- Flat CFR ---
pivot_ports_tons["CFR_LQW_Santos"] = (pivot_ports_tons["LQW_Santos"] + NY_Price).round(1)
pivot_ports_tons["CFR_LQW_India"]  = (pivot_ports_tons["LQW_India"]  + NY_Price).round(1)

pivot_ports_tons["CFR_HQW_Santos"]       = (pivot_ports_tons["HQW_Santos"]       + LDN_Price).round(1)
pivot_ports_tons["CFR_HQW_India"]        = (pivot_ports_tons["HQW_India"]        + LDN_Price).round(1)
pivot_ports_tons["CFR_HQW_Antwerp"]      = (pivot_ports_tons["HQW_Antwerp"]      + LDN_Price).round(1)
pivot_ports_tons["CFR_HQW_Jabel_Ali"]    = (pivot_ports_tons["HQW_Jabel_Ali"]    + LDN_Price).round(1)
pivot_ports_tons["CFR_HQW_Laem_Chabang"] = (pivot_ports_tons["HQW_Laem_Chabang"] + LDN_Price).round(1)

# --- Premiums ---
premiums = {
    "CFR_LQW_Santos":        104,
    "CFR_LQW_India":         120,
    "CFR_HQW_Santos":         20,
    "CFR_HQW_India":          30,
    "CFR_HQW_Antwerp":        20,
    "CFR_HQW_Jabel_Ali":      40,
    "CFR_HQW_Laem_Chabang":   30
}

for col, prem in premiums.items():
    pivot_ports_tons[col] = (pivot_ports_tons[col] + prem).round(1)

display(pivot_ports_tons)


['Region', 'Country POD', 'Port POD', 'Santos', 'Mundra', 'Antwerp', 'Jabel Ali', 'Laem Chabang', 'Cheapest_POL', 'LQW_Santos', 'HQW_Santos', 'HQW_India', 'LQW_India', 'HQW_Antwerp', 'HQW_Jabel_Ali', 'HQW_Laem_Chabang', 'Cheapest_Origin', 'Cheapest_Origin_Value', 'CFR_LQW_Santos', 'CFR_LQW_India', 'CFR_HQW_Santos', 'CFR_HQW_India', 'CFR_HQW_Antwerp', 'CFR_HQW_Jabel_Ali', 'CFR_HQW_Laem_Chabang', 'Cheapest_CFR_Origin', 'Cheapest_CFR_Value', 'Cheapest_CFR_Origin_Name']


,Region,Country POD,Port POD,Santos,Mundra,Antwerp,Jabel Ali,Laem Chabang,Cheapest_POL,LQW_Santos,HQW_Santos,HQW_India,LQW_India,HQW_Antwerp,HQW_Jabel_Ali,HQW_Laem_Chabang,Cheapest_Origin,Cheapest_Origin_Value,CFR_LQW_Santos,CFR_LQW_India,CFR_HQW_Santos,CFR_HQW_India,CFR_HQW_Antwerp,CFR_HQW_Jabel_Ali,CFR_HQW_Laem_Chabang,Cheapest_CFR_Origin,Cheapest_CFR_Value,Cheapest_CFR_Origin_Name,Second_Cheapest_CFR_Origin,Second_Cheapest_CFR_Value
3,Africa West,Benin,Cotonou,1527.0,2317.0,1731.0,2956.0,1531.0,Santos,56.6,58.7,85.8,85.8,72.1,113.7,58.9,LQW_Santos,56.6,456.9,502.1,474.7,511.8,488.1,549.7,484.9,CFR_LQW_Santos,456.9,Santos LQW,CFR_HQW_Santos,474.7
4,Africa West,Cameroon,Douala,1651.0,3865.0,1952.0,3998.0,2031.0,Santos,61.1,63.5,143.1,143.1,81.3,153.8,78.1,LQW_Santos,61.1,461.4,559.4,479.5,569.1,497.3,589.8,504.1,CFR_LQW_Santos,461.4,Santos LQW,CFR_HQW_Santos,479.5
7,Africa West,Ghana,Tema,1451.0,2725.0,1214.0,3245.0,1599.0,Antwerp,53.7,55.8,100.9,100.9,50.6,124.8,61.5,HQW_Antwerp,50.6,454.0,517.2,471.8,526.9,466.6,560.8,487.5,CFR_LQW_Santos,454.0,Santos LQW,CFR_HQW_Antwerp,466.6
8,Africa West,Guinea,Conakry,3851.0,7204.0,4869.0,6088.0,4982.0,Santos,142.6,148.1,266.8,266.8,202.9,234.2,191.6,LQW_Santos,142.6,542.9,683.1,564.1,692.8,618.9,670.2,617.6,CFR_LQW_Santos,542.9,Santos LQW,CFR_HQW_Santos,564.1
9,Africa West,Ivory Coast,Abidjan,1627.0,2404.0,1080.0,3012.0,1531.0,Antwerp,60.3,62.6,89.0,89.0,45.0,115.8,58.9,HQW_Antwerp,45.0,460.6,505.3,478.6,515.0,461.0,551.8,484.9,CFR_LQW_Santos,460.6,Santos LQW,CFR_HQW_Antwerp,461.0
12,Africa West,Senegal,Dakar,1627.0,3599.0,1876.0,3018.0,2781.0,Santos,60.3,62.6,133.3,133.3,78.2,116.1,107.0,LQW_Santos,60.3,460.6,549.6,478.6,559.3,494.2,552.1,533.0,CFR_LQW_Santos,460.6,Santos LQW,CFR_HQW_Santos,478.6
15,Africa West,Togo,Lome,1490.0,2370.0,1589.0,2891.0,1531.0,Santos,55.2,57.3,87.8,87.8,66.2,111.2,58.9,LQW_Santos,55.2,455.5,504.1,473.3,513.8,482.2,547.2,484.9,CFR_LQW_Santos,455.5,Santos LQW,CFR_HQW_Santos,473.3
19,Europe,Albania,Durres,1647.0,2557.0,1292.0,1909.0,5225.0,Antwerp,61.0,63.3,94.7,94.7,53.8,73.4,201.0,HQW_Antwerp,53.8,461.3,511.0,479.3,520.7,469.8,509.4,627.0,CFR_LQW_Santos,461.3,Santos LQW,CFR_HQW_Antwerp,469.8
24,Far East / South East Asia,Singapore,Singapore,941.0,624.0,18.0,282.0,350.0,Antwerp,34.9,36.2,23.1,23.1,0.8,10.8,13.5,HQW_Antwerp,0.8,435.2,439.4,452.2,449.1,416.8,446.8,439.5,CFR_HQW_Antwerp,416.8,Antwerp HQW,CFR_LQW_Santos,435.2
25,Far East / South East Asia,Sri Lanka,Colombo,750.0,1869.0,491.0,339.0,1817.0,Jabel Ali,27.8,28.8,69.2,69.2,20.5,13.0,69.9,HQW_Jabel_Ali,13.0,428.1,485.5,444.8,495.2,436.5,449.0,495.9,CFR_LQW_Santos,428.1,Santos LQW,CFR_HQW_Antwerp,436.5


In [30]:
cfr_cols = [
    "CFR_LQW_Santos",
    "CFR_LQW_India",
    "CFR_HQW_Santos",
    "CFR_HQW_India",
    "CFR_HQW_Antwerp",
    "CFR_HQW_Jabel_Ali",
    "CFR_HQW_Laem_Chabang"
]

pivot_ports_tons["Cheapest_CFR_Origin"] = (
    pivot_ports_tons[cfr_cols]
    .idxmin(axis=1)
)

pivot_ports_tons["Cheapest_CFR_Value"] = (
    pivot_ports_tons[cfr_cols]
    .min(axis=1)
    .round(1)
)

pivot_ports_tons["Second_Cheapest_CFR_Origin"] = (
    pivot_ports_tons[cfr_cols]
    .apply(lambda r: r.nsmallest(2).index[-1], axis=1)
)

pivot_ports_tons["Second_Cheapest_CFR_Value"] = (
    pivot_ports_tons[cfr_cols]
    .apply(lambda r: r.nsmallest(2).iloc[-1], axis=1)
    .round(1)
)

origin_map = {
    "CFR_LQW_Santos":        "Santos LQW",
    "CFR_LQW_India":         "India LQW",
    "CFR_HQW_Santos":        "Santos HQW",
    "CFR_HQW_India":         "India HQW",
    "CFR_HQW_Antwerp":       "Antwerp HQW",
    "CFR_HQW_Jabel_Ali":     "Jebel Ali HQW",
    "CFR_HQW_Laem_Chabang":  "Laem Chabang HQW"
}

pivot_ports_tons["Cheapest_CFR_Origin_Name"] = (
    pivot_ports_tons["Cheapest_CFR_Origin"]
    .map(origin_map)
)





display(pivot_ports_tons)


,Region,Country POD,Port POD,Santos,Mundra,Antwerp,Jabel Ali,Laem Chabang,Cheapest_POL,LQW_Santos,HQW_Santos,HQW_India,LQW_India,HQW_Antwerp,HQW_Jabel_Ali,HQW_Laem_Chabang,Cheapest_Origin,Cheapest_Origin_Value,CFR_LQW_Santos,CFR_LQW_India,CFR_HQW_Santos,CFR_HQW_India,CFR_HQW_Antwerp,CFR_HQW_Jabel_Ali,CFR_HQW_Laem_Chabang,Cheapest_CFR_Origin,Cheapest_CFR_Value,Cheapest_CFR_Origin_Name,Second_Cheapest_CFR_Origin,Second_Cheapest_CFR_Value
3,Africa West,Benin,Cotonou,1527.0,2317.0,1731.0,2956.0,1531.0,Santos,56.6,58.7,85.8,85.8,72.1,113.7,58.9,LQW_Santos,56.6,456.9,502.1,474.7,511.8,488.1,549.7,484.9,CFR_LQW_Santos,456.9,Santos LQW,CFR_HQW_Santos,474.7
4,Africa West,Cameroon,Douala,1651.0,3865.0,1952.0,3998.0,2031.0,Santos,61.1,63.5,143.1,143.1,81.3,153.8,78.1,LQW_Santos,61.1,461.4,559.4,479.5,569.1,497.3,589.8,504.1,CFR_LQW_Santos,461.4,Santos LQW,CFR_HQW_Santos,479.5
7,Africa West,Ghana,Tema,1451.0,2725.0,1214.0,3245.0,1599.0,Antwerp,53.7,55.8,100.9,100.9,50.6,124.8,61.5,HQW_Antwerp,50.6,454.0,517.2,471.8,526.9,466.6,560.8,487.5,CFR_LQW_Santos,454.0,Santos LQW,CFR_HQW_Antwerp,466.6
8,Africa West,Guinea,Conakry,3851.0,7204.0,4869.0,6088.0,4982.0,Santos,142.6,148.1,266.8,266.8,202.9,234.2,191.6,LQW_Santos,142.6,542.9,683.1,564.1,692.8,618.9,670.2,617.6,CFR_LQW_Santos,542.9,Santos LQW,CFR_HQW_Santos,564.1
9,Africa West,Ivory Coast,Abidjan,1627.0,2404.0,1080.0,3012.0,1531.0,Antwerp,60.3,62.6,89.0,89.0,45.0,115.8,58.9,HQW_Antwerp,45.0,460.6,505.3,478.6,515.0,461.0,551.8,484.9,CFR_LQW_Santos,460.6,Santos LQW,CFR_HQW_Antwerp,461.0
12,Africa West,Senegal,Dakar,1627.0,3599.0,1876.0,3018.0,2781.0,Santos,60.3,62.6,133.3,133.3,78.2,116.1,107.0,LQW_Santos,60.3,460.6,549.6,478.6,559.3,494.2,552.1,533.0,CFR_LQW_Santos,460.6,Santos LQW,CFR_HQW_Santos,478.6
15,Africa West,Togo,Lome,1490.0,2370.0,1589.0,2891.0,1531.0,Santos,55.2,57.3,87.8,87.8,66.2,111.2,58.9,LQW_Santos,55.2,455.5,504.1,473.3,513.8,482.2,547.2,484.9,CFR_LQW_Santos,455.5,Santos LQW,CFR_HQW_Santos,473.3
19,Europe,Albania,Durres,1647.0,2557.0,1292.0,1909.0,5225.0,Antwerp,61.0,63.3,94.7,94.7,53.8,73.4,201.0,HQW_Antwerp,53.8,461.3,511.0,479.3,520.7,469.8,509.4,627.0,CFR_LQW_Santos,461.3,Santos LQW,CFR_HQW_Antwerp,469.8
24,Far East / South East Asia,Singapore,Singapore,941.0,624.0,18.0,282.0,350.0,Antwerp,34.9,36.2,23.1,23.1,0.8,10.8,13.5,HQW_Antwerp,0.8,435.2,439.4,452.2,449.1,416.8,446.8,439.5,CFR_HQW_Antwerp,416.8,Antwerp HQW,CFR_LQW_Santos,435.2
25,Far East / South East Asia,Sri Lanka,Colombo,750.0,1869.0,491.0,339.0,1817.0,Jabel Ali,27.8,28.8,69.2,69.2,20.5,13.0,69.9,HQW_Jabel_Ali,13.0,428.1,485.5,444.8,495.2,436.5,449.0,495.9,CFR_LQW_Santos,428.1,Santos LQW,CFR_HQW_Antwerp,436.5


In [23]:
print(pivot_ports_tons.columns.tolist())

['Region', 'Country POD', 'Port POD', 'Santos', 'Mundra', 'Antwerp', 'Jabel Ali', 'Laem Chabang', 'Cheapest_POL', 'LQW_Santos', 'HQW_Santos', 'HQW_India', 'LQW_India', 'HQW_Antwerp', 'HQW_Jabel_Ali', 'HQW_Laem_Chabang', 'Cheapest_Origin', 'Cheapest_Origin_Value', 'CFR_LQW_Santos', 'CFR_LQW_India', 'CFR_HQW_Santos', 'CFR_HQW_India', 'CFR_HQW_Antwerp', 'CFR_HQW_Jabel_Ali', 'CFR_HQW_Laem_Chabang', 'Cheapest_CFR_Origin', 'Cheapest_CFR_Value', 'Cheapest_CFR_Origin_Name']


In [26]:
pivot_ports_tons_cfr = pivot_ports_tons[['Region', 'Country POD', 'Port POD', 
                                         'CFR_LQW_Santos', 'CFR_LQW_India', 'CFR_HQW_Santos', 
                                         'CFR_HQW_India', 'CFR_HQW_Antwerp', 'CFR_HQW_Jabel_Ali', 
                                         'CFR_HQW_Laem_Chabang', 'Cheapest_CFR_Origin_Name', 'Cheapest_CFR_Value']].copy()

pivot_ports_tons_cfr = pivot_ports_tons_cfr.reset_index(drop=True)

display(pivot_ports_tons_cfr)

,Region,Country POD,Port POD,CFR_LQW_Santos,CFR_LQW_India,CFR_HQW_Santos,CFR_HQW_India,CFR_HQW_Antwerp,CFR_HQW_Jabel_Ali,CFR_HQW_Laem_Chabang,Cheapest_CFR_Origin_Name,Cheapest_CFR_Value
0,Africa West,Benin,Cotonou,456.9,502.1,474.7,511.8,488.1,549.7,484.9,Santos LQW,456.9
1,Africa West,Cameroon,Douala,461.4,559.4,479.5,569.1,497.3,589.8,504.1,Santos LQW,461.4
2,Africa West,Ghana,Tema,454.0,517.2,471.8,526.9,466.6,560.8,487.5,Santos LQW,454.0
3,Africa West,Guinea,Conakry,542.9,683.1,564.1,692.8,618.9,670.2,617.6,Santos LQW,542.9
4,Africa West,Ivory Coast,Abidjan,460.6,505.3,478.6,515.0,461.0,551.8,484.9,Santos LQW,460.6
5,Africa West,Senegal,Dakar,460.6,549.6,478.6,559.3,494.2,552.1,533.0,Santos LQW,460.6
6,Africa West,Togo,Lome,455.5,504.1,473.3,513.8,482.2,547.2,484.9,Santos LQW,455.5
7,Europe,Albania,Durres,461.3,511.0,479.3,520.7,469.8,509.4,627.0,Santos LQW,461.3
8,Far East / South East Asia,Singapore,Singapore,435.2,439.4,452.2,449.1,416.8,446.8,439.5,Antwerp HQW,416.8
9,Far East / South East Asia,Sri Lanka,Colombo,428.1,485.5,444.8,495.2,436.5,449.0,495.9,Santos LQW,428.1


In [27]:
import os
from datetime import datetime

# -----------------------------
# Save path
# -----------------------------
path_save = r"C:\Users\djenn\OneDrive - Sopex London Ltd\Shared\22.Databasis\2_output\2_Cash_prices"

os.makedirs(path_save, exist_ok=True)

# -----------------------------
# File name (YYYYMMDD)
# -----------------------------
today_str = datetime.today().strftime("%Y%m%d")

file_name = f"{today_str}_freight_matrix.xlsx"
full_path = os.path.join(path_save, file_name)

# -----------------------------
# Export to Excel
# -----------------------------
pivot_ports_tons.to_excel(
    full_path,
    index=False,
    engine="openpyxl"
)

print(f"File saved here: {full_path}")


File saved here: C:\Users\djenn\OneDrive - Sopex London Ltd\Shared\22.Databasis\2_output\2_Cash_prices\20260213_freight_matrix.xlsx


In [ ]:




# Format date
today = datetime.today().strftime("%Y%m%d")

# Build full path + filename
path = r"A:\OneDrive - Sopex London Ltd\Shared\19. Shipments - Sopex"
filename = f"{today}_last_freight.xlsx"

full_path = f"{path}\\{filename}"

# Save
df_future.to_excel(full_path, index=False)

In [ ]:
# Format date
today = datetime.today().strftime("%Y%m%d")

# Build full path + filename
path = r"A:\OneDrive - Sopex London Ltd\Shared\19. Shipments - Sopex"
filename = f"{today}_last_freight.xlsx"

full_path = f"{path}\\{filename}"

# Save
df_future.to_excel(full_path, index=False)